In [558]:
%run solve_vinet.ipynb
%run solve_bm3.ipynb
%run solve_murnaghan.ipynb
%run solve_holzapfel.ipynb
%run solve_adams_williamson.ipynb
%run ice_ball_dictionary.ipynb
%run EoS_Bits.ipynb

G = 6.674 / (10 ** 11)
r_earth = 6.371 * (10 ** 6)
r_moon = 1.737 * (10 ** 6)
r_mars = 3.390 * (10 ** 6)
#For water
rho0 = 1487
B0 = 14.9 * (10 ** 9)
B01 = 6.2

def Solver(radii_list, insert_dict, density_list = [], discrepancy = 10, calls = 0):
#starts with no density_list
    pressure_n = {}
    pressure_n1 = {}
    density_n1 = {}
    calls += 1


    while calls <= 1500:
        if int(max([*insert_dict.keys()])) != int(max(radii_list)):
            insert_dict[int(float(f"{int(max(radii_list))}"))] = insert_dict.pop(int(float(f'{max([*insert_dict.keys()])}')))
            return(Solver(radii_list, insert_dict, density_list = [], discrepancy = discrepancy, calls = 0))
            #return(max([*insert_dict.keys()]), int(max(radii_list)), 'The highest radius given in the radii_list does not match the highest radius in the density profile dictionary.')
    #checks to make sure input lengths are correct
        
        if len(density_list) == 0:
            for count in range(len(radii_list)):
                for cutoff in [*insert_dict.keys()]:
                        if radii_list[count] <= cutoff:
                            if count == len(density_list):
                                density_list.append(int(insert_dict[cutoff][0]))
    #makes a density_list using cutoffs as preliminary values
    
    
        a_w = adams_williamson(radii_list, density_list)
        
        for n in radii_list:
            pressure_n[n] = a_w[n][1]
    #now have pressures for each radius
        
        for n in radii_list:
            nth_p_n = pressure_n[n]
            for cutoff in list(insert_dict.keys()):
    #for each piecewise cutoff in our dictionary, we have different EOS parameters
                if n <= cutoff: 
    
                    if list(insert_dict.keys()).index(cutoff) == 0:

                        density_n1[n] = CoreDensity(nth_p_n)
                        pressure_n1[n] = Vinet(density_n1[n],7678,136.2 * (10 ** 9),5.97)
                        break
                        
                    elif list(insert_dict.keys()).index(cutoff) == 1:

                        density_n1[n] = RockDensity(nth_p_n)
    
                        if nth_p_n < 2.5 * (10 ** 9):
                            pressure_n1[n] = 0.701 * BM3(density_n1[n], 3221,125 * (10 ** 9),4) + 0.299 * BM3(density_n1[n],2648,37.4 * (10 ** 9),6.2)
                            break
                        elif nth_p_n < 8 * (10 ** 9):
                            pressure_n1[n] = 0.701 * BM3(density_n1[n], 3221,125 * (10 ** 9),4) + 0.299 * BM3(density_n1[n],2921,96 * (10 ** 9),8.4)
                            break
                        elif nth_p_n < 14 * (10 ** 9):
                            pressure_n1[n] = 0.701 * BM3(density_n1[n], 3221,125 * (10 ** 9),4) + 0.299 * BM3(density_n1[n],4290,309.9 * (10 ** 9),4.59)
                            break
                        elif nth_p_n < 18 * (10 ** 9):
                            pressure_n1[n] = 0.701 * BM3(density_n1[n],3491,160 * (10 ** 9),4) + 0.299 * BM3(density_n1[n],4290,309.9 * (10 ** 9),4.59)
                            break
                        elif nth_p_n < 23 * (10 ** 9):
                            pressure_n1[n] = 0.701 * BM3(density_n1[n],3548,182 * (10 ** 9),4.2) + 0.299 * BM3(density_n1[n],4290,309.9 * (10 ** 9),4.59)
                            break
                        elif nth_p_n < 120 * (10 ** 9):
                            pressure_n1[n] = BM3(density_n1[n], 4101,256 * (10 ** 9),4)
                            break
                        else:
                            pressure_n1[n] = Vinet(density_n1[n], 4058,221 * (10 ** 9),4.2)
                            break
                        
                    elif list(insert_dict.keys()).index(cutoff) == 2:
                        density_n1[n] = IceDensity(nth_p_n)

                        if nth_p_n < 1 * (10 ** 9):
                            pressure_n1[n] = Murnaghan(density_n1[n], 930, 9.85 * (10 ** 9), 6.6)
                            break
                        elif nth_p_n < 2.1 * (10 ** 9):
                            pressure_n1[n] = BM3(density_n1[n], 1271, 14.05 * (10 ** 9), 4)
                            break
                        else:
                            pressure_n1[n] = BM3(density_n1[n], 1456, 14.9 * (10 ** 9), 5.4)
                            break
    
        if all(abs(np.array(list(density_n1.values())[:1000]) - np.array(density_list[:1000]) < discrepancy)):
            #print('number of calls:', calls)
            #print('list of pressuren1', list(pressure_n.values()))
            return(list(density_n1.values()), list(pressure_n1.values()))
    
        else:
            #print('loop length of density', len(list(density_n1.values())))
            return(Solver(radii_list, insert_dict, density_list = list(density_n1.values()), discrepancy = discrepancy, calls = calls))
    return([0], [0])

#--------------------------------------------------

import matplotlib.pyplot as plt

def plotter(radii_list, insert_dict, density_or_pressure):
    if density_or_pressure.lower() == "density":
        to_plot = Solver(radii_list, insert_dict)[0]
    elif density_or_pressure.lower() == "pressure":
        to_plot = Solver(radii_list, insert_dict)[1]
    plt.plot(radii_list, to_plot)
    plt.show()

#--------------------------------------------------
print(ice_ball_dictionary(1, 0.5, 0.5))
Solver([*range(1, int(r_earth), int(r_earth / 1000))], ice_ball_dictionary(1, 0.5, 0.5))
#plotter([*range(1, int(r_earth), 6371)], ice_ball_dictionary(1, 0.3, 0.5), 'density')
#plotter([*range(1, int(r_earth), 6371)], ice_ball_dictionary(1, 0.3, 0.5), 'pressure')


{3185500: [7678], 6371000: [930]}


([13633.294000190646,
  13633.241547030495,
  13633.175978399508,
  13633.094381782332,
  13632.996028720449,
  13632.880627631966,
  13632.74803253769,
  13632.598159831814,
  13632.430957067958,
  13632.246389085787,
  13632.044431074704,
  13631.825064789566,
  13631.58827634357,
  13631.334054849645,
  13631.06239154774,
  13630.773279221892,
  13630.466711800666,
  13630.142684074024,
  13629.801191489649,
  13629.442230002445,
  13629.065795961462,
  13628.671886024433,
  13628.26049709096,
  13627.83162625074,
  13627.385270742081,
  13626.921427919322,
  13626.440095226004,
  13625.941270173098,
  13625.424950321532,
  13624.891133266654,
  13624.339816626867,
  13623.770998032578,
  13623.184675117674,
  13622.580845511959,
  13621.959506835392,
  13621.32065669186,
  13620.66429266459,
  13619.990412312589,
  13619.299013166867,
  13618.590092727012,
  13617.863648458522,
  13617.11967779088,
  13616.35817811481,
  13615.579146780707,
  13614.782581097192,
  13613.96847832872

true
